In [ ]:
import theano
import lasagne
import pandas as pd
import numpy as np

from nltk.corpus import stopwords

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from nolearn.lasagne import NeuralNet
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import BatchIterator
from lasagne.layers import InputLayer, Conv2DLayer, DropoutLayer,\
    MaxPool2DLayer, DenseLayer
from lasagne.nonlinearities import softmax, sigmoid
from sklearn.cross_validation import train_test_split

In [ ]:
df = pd.read_csv('data/imdb.csv')

In [ ]:
def agrupar_pelis(x):
    if x >=8:
        return 2
    elif x<8 and x>=6:
        return 1
    else:
        return 0

df['imdb_score'] = df['imdb_score'].apply(agrupar_pelis)

In [ ]:
ratings = np.sort(df['imdb_score'].unique())

In [ ]:
len(df[df.imdb_score == 0]), len(df[df.imdb_score == 1]),len(df[df.imdb_score == 2])

In [ ]:
#df['earning_rate'] = df.apply(lambda r: r.gross / r.budget, axis=1)

In [ ]:
a = df.corr()

In [ ]:
import matplotlib.pyplot as plt

df_corr = df[a.columns.tolist()]

plt.matshow(df_corr.corr())
plt.xticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist(), rotation="vertical")
plt.yticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist())
plt.colorbar()
plt.show()

In [ ]:
ac1 = df['actor_1_name'].unique()
ac2 = df['actor_2_name'].unique()
ac3 = df['actor_3_name'].unique()

In [ ]:
total = np.concatenate((ac1, ac2, ac3))

In [ ]:
total = dict(enumerate(set(total)))

In [ ]:
def factorizar(row):
    for x in total:
        if total[x] == row:
            return int(x)

In [ ]:
df['actor_1_name'] = df['actor_1_name'].apply(lambda row: factorizar(row))
df['actor_2_name'] = df['actor_2_name'].apply(lambda row: factorizar(row))
df['actor_3_name'] = df['actor_3_name'].apply(lambda row: factorizar(row))

In [ ]:
## Para obtener los indices de los paises y los lenguajes de las peliculas
idx_language, label_language = pd.factorize(df['language'])
idx_country, label_country = pd.factorize(df['country'])

## Para obtener los indices de los actores
#idx_actor1, label_actor1 = pd.factorize(df['actor_1_name'])
#idx_actor2, label_actor2 = pd.factorize(df['actor_2_name'])
#idx_actor3, label_actor3 = pd.factorize(df['actor_3_name'])

## Para obtener el indice del director
idx_director, label_director = pd.factorize(df['director_name'])

#Para obtener los indices del titulo de las peliculas y el content rating
idx_movie, label_movie = pd.factorize(df['movie_title'])
idx_content, label_content = pd.factorize(df['content_rating'])

In [ ]:
## Separa los generos en columnas
df = pd.concat([df, df.genres.str.get_dummies(sep='|')], axis=1)

In [ ]:
df_separados = pd.DataFrame()
df_separados = pd.concat([df_separados, df.plot_keywords.str.replace('|',' ')],axis=1)

stopw = set(stopwords.words('english')) 
corpus = []
for e in df_separados.values:
    if type(e[0]) is str:
        corpus.append(e[0])
corpus1 = " ".join(corpus)
corpus1 = corpus1.split(" ")

In [ ]:
corpus1 = list(filter(lambda x: not x[0] in stopw, corpus1))
corpus1 = set(corpus1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vocabulary = corpus1
vect = TfidfVectorizer(sublinear_tf=True, max_df=2, analyzer='word', 
           stop_words=stopw, vocabulary=vocabulary)
vect.fit(corpus)
#print(dict(zip(vect.get_feature_names(), vect.idf_)))

In [ ]:
def transformar_keywords(k):
    if not type(k) is str:
        return 0  
    keywords = k.replace('|'," ")
    doc_tfidf = vect.transform([k])
    t = doc_tfidf.toarray().sum()
    return t

In [ ]:
df['plot_keywords'] = df['plot_keywords'].apply(lambda row: transformar_keywords(row)) 

In [ ]:
df['language'] = idx_language
df['country'] = idx_country
#df['actor_1_name'] = idx_actor1
#df['actor_2_name'] = idx_actor2
#df['actor_3_name'] = idx_actor3
df['director_name'] = idx_director
df['movie_title'] = idx_movie
df['content_rating'] = idx_content

In [ ]:
df.fillna(value=-1, inplace=True)

In [ ]:
ys = df['imdb_score']
df.drop('imdb_score', axis=1, inplace=True)

### Columnas Dropeadas

In [ ]:
df.columns

In [ ]:
#df.drop('actor_1_facebook_likes', axis=1, inplace=True)
#df.drop('actor_2_facebook_likes', axis=1, inplace=True)
#df.drop('actor_3_facebook_likes', axis=1, inplace=True)
#df.drop('director_facebook_likes', axis=1, inplace=True)

#df.drop('director_name', axis=1, inplace=True)
#df.drop('actor_1_name', axis=1, inplace=True)
#df.drop('actor_2_name', axis=1, inplace=True)
#df.drop('actor_3_name', axis=1, inplace=True)

#df.drop('cast_total_facebook_likes', axis=1, inplace=True)
#df.drop('movie_facebook_likes', axis=1, inplace=True)
#df.drop('budget', axis=1, inplace=True)
#df.drop('gross', axis=1, inplace=True)
#df.drop('cast_total_facebook_likes', axis=1, inplace=True)
#df.drop('plot_keywords', axis=1, inplace=True)
df.drop('genres', axis=1, inplace=True)
df.drop('movie_imdb_link', axis=1, inplace=True)
df.drop('color', axis=1, inplace=True)
#df.drop('num_user_for_reviews', axis=1, inplace=True)
#df.drop('movie_title', axis=1, inplace=True)
#df.drop('content_rating', axis=1, inplace=True)
#df.drop('duration', axis=1, inplace=True)

In [ ]:
xs = df.values

In [ ]:
std_x = MinMaxScaler([-1,1])
xs = std_x.fit_transform(xs.astype(np.float32))
std_y = MinMaxScaler([-1, 1])
ys = std_y.fit_transform(ys)

In [ ]:
ys = np.array(ys)

In [ ]:
a = df.corr()

df_corr = df[a.columns.tolist()]

plt.matshow(df_corr.corr())
plt.xticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist(), rotation="vertical")
plt.yticks(np.arange(0, len(df_corr.columns)), df_corr.columns.tolist())
plt.colorbar()
plt.show()

# PCA

In [ ]:
from sklearn.decomposition import RandomizedPCA # using randomized Singular Value Decomposition 
Xp = RandomizedPCA(n_components=35, random_state=1)
Xp = Xp.fit_transform(xs)

#std_x = MinMaxScaler([-1,1])
#xs = std_x.fit_transform(Xp)
#std_y = MinMaxScaler([-1,1])
#ys = std_y.fit_transform(ys)

In [ ]:
#Xp.explained_variance_ratio_.cumsum()

In [ ]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(xs, ys,random_state=1)

In [ ]:
y_train

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics


def plot_matrix(clf, X_test, y_test):
    plt.clf()
    plt.imshow(confusion_matrix(clf.predict(X_test), y_test),
               interpolation='nearest', cmap=plt.cm.Blues)
    plt.colorbar()
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    plt.show()

clf = ExtraTreesClassifier(n_estimators=300,
                           max_features=0.2, 
                           n_jobs=2,
                           max_depth=None,
                           min_samples_split=1,
                           random_state=1).fit(X_train, y_train)
print(classification_report(clf.predict(X_test), y_test))
print("Score over Testing Data {}".format(clf.score(X_test, y_test)))
print("Score over Training Data {}".format(clf.score(X_train, y_train)))
plot_matrix(clf, X_test, y_test)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
importances = clf.feature_importances_
cols = len(df.columns)

text = list(map(lambda i: df.columns[i], range(cols)))
plt.figure(figsize=(20,cols))
print(importances[::-1].shape)
plt.bar(range(cols),height=importances,  width=1.)
plt.xticks(np.arange(0.5, cols, 1.), text, rotation=90)
plt.xlim((0, cols))
#plt.show()

In [ ]:
confusion_matrix(clf.predict(X_test), y_test)

## Dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(random_state=0).fit(X_train, y_train)
#print clf.score(X_test, y_test)
plot_matrix(clf, X_test, y_test)
clf.score(X_test, y_test)
print(classification_report(clf.predict(X_test), y_test))

## SVM

In [ ]:
from sklearn.svm import SVC

sv = SVC(kernel='rbf', cache_size=1000)
sv.fit(X_train, y_train)

print(classification_report(sv.predict(X_test), y_test))
print(sv.score(X_test, y_test))
plot_matrix(sv, X_test, y_test)

# Red Neuronal

In [ ]:
from lasagne.nonlinearities import linear, tanh

In [ ]:
layers_0 = [
                (InputLayer, {'shape': (None, 50)}),
                (DenseLayer, {'num_units': 500}),
                (DropoutLayer, {'p': 0.5}),
                (DenseLayer, {'num_units': 300}),
                (DropoutLayer, {'p': 0.3}),
                (DenseLayer, {'num_units': 3, 'nonlinearity': softmax}),
        ]

In [ ]:
class AdjustVariable(object):
    """
    Used to decreases linearly the learning rate with the number of epochs,
    while we the momentum increase.
    """
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = np.float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)

In [ ]:
def create_network(npochs=50, batch_s=10000):
    return NeuralNet(
        layers=layers_0,
        update=nesterov_momentum,
        update_learning_rate=theano.shared(np.float32(0.0009)),
        update_momentum=theano.shared(np.float32(0.9)),

        regression=False,
        batch_iterator_train=BatchIterator(batch_size=batch_s),
        on_epoch_finished=[
            AdjustVariable('update_learning_rate', start=0.009, stop=0.0009),
            AdjustVariable('update_momentum', start=0.9, stop=0.9999)
        ],
        max_epochs=npochs,
        verbose=1)

net0 = create_network(150)

In [ ]:
#theano.config.profile = True


In [ ]:
y_train = y_train.astype(np.int32)
X_train = X_train.astype(np.float32)

net0.fit(X_train, y_train)

In [ ]:
%matplotlib inline

plt.clf()
plt.figure(figsize=(15,5))

train_loss = np.array([i["train_loss"] for i in net0.train_history_])
valid_loss = np.array([i["valid_loss"] for i in net0.train_history_])
plt.plot(train_loss, '--b', linewidth=2, label="{} train".format("net0"))
plt.plot(valid_loss, '-b', linewidth=2, label="{} valid".format("net0"))

plt.grid()
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.yscale("log")
plt.show()

In [ ]:
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

print(classification_report(net0.predict(X_test), y_test))
plot_matrix(net0, X_test, y_test)

# <span style="color:red"> VER ESTO!! NO ESTA TOMANDO LOS ELEMENTOS DE LAS CLASES 0 Y 2</span>.

In [ ]:
confusion_matrix(net0.predict(X_test), y_test)

In [ ]:
len(y_test)

In [ ]:
set(y_test)

In [ ]:
246 + 733 + 83

In [ ]:
168 + 93 + 4

In [ ]:
df = pd.DataFrame(y_test, columns=["cero"])

In [ ]:
len(df[df.cero == 2])